In [1]:
import math
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
pd.set_option('max_rows',200)
pd.set_option('max_columns',200)
warnings.filterwarnings('ignore')
from itertools import combinations  
from bisect import bisect_left, bisect_right

In [2]:
data = pd.read_csv('converted.csv')
data.rename(columns={'Test#2: Технології прикладного програмування':'Test#2: Гуманітарний предмет'}, inplace=True) ##remove this line
data = data[['Test#1', 'Предмет#2', 'Назва предмету, який вважаєш проф-орієнтованим', 'Test#2: Гуманітарний предмет', 'Test#2: Аналіз даних', 
             'Курс', 'Професійні мотиви', 'Соціальні мотиви']]
data.head()

Number of respondents:  103


,Test#1,Предмет#2,"Назва предмету, який вважаєш проф-орієнтованим",Test#2: Гуманітарний предмет,Test#2: Аналіз даних,Курс,Професійні мотиви,Соціальні мотиви
0,13,81,мат. аналіз,9,13,1,3.6667,2.4
1,15,94,дискр. математика,7,13,1,3.1667,2.6
2,14,91,програмування,9,9,1,3.3333,3.0
3,9,91,програмування,8,8,1,3.1667,3.0
4,14,77,мат. аналіз,9,10,1,3.6667,2.4


In [3]:
math_sub = ['мат. аналіз', 'лінійна алгебра','ятдр','дискр. математика','математична економіка', 'статистичні основи вебаналітики', 'обчислювальна геометрія', 'тймс', 'любий предмет міхалича']
test_1_data = data.loc[data['Назва предмету, який вважаєш проф-орієнтованим'].isin(math_sub)][['Test#1', 'Предмет#2']]

# 1. Знайти вибіркові коефіцієнту кореляції 
 - Пірсона
 - Спірмена
 - Кенделла 

та перевірити їхню значущість за допомогою параметричного критерію Пірсона та непараметричних Спірмена та Кенделла.

In [4]:
def mean(x):
    sum = 0.0
    for i in x:
         sum += i
    return sum / len(x) 

def std(x):
    sum_v = 0.0
    for i in x:
         sum_v += (i - mean(x))**2
    return math.sqrt(sum_v/ (len(x) - 1))

def covariance(x, y):
    X = np.column_stack([x, y])
    X -= X.mean(axis=0) 
    fact = len(x) - 1 
    cov = np.dot(X.T, X.conj()) / fact
    return cov
    
def pearson_pvalue(n, r):
    t = (r * math.sqrt(n - 2)) / (math.sqrt(1 - r ** 2))
    res = stats.t.cdf(t, n - 2)
    return res
    
def pearson(x, y):
    res_x = []
    res_y = []
    for i in x: 
        val_x = (i - mean(x)) / std(x)
        res_x.append(val_x) 
    for j in y:
        val_y = (j - mean(y)) / std(y)
        res_y.append(val_y)
    res = (sum([i*j for i,j in zip(res_x,res_y)]))/(len(x)-1)
    return res, pearson_pvalue(len(x), res)

def spearman(x, y):
    rank_x = stats.rankdata(x)
    rank_y = stats.rankdata(y)
    cov = covariance(rank_x, rank_y)[0][1]
    res = cov / (std(rank_x) * std(rank_y))
    return res, None

def kendall(x, y):
    conc, disc, ties = 0, 0, 0
    for i in range(len(x)):
        for j in range(len(y)):
            if ((x[i] > x[j] and y[i] > y[j]) or (x[i] < x[j] and y[i] < y[j])):
                conc += 1
            elif ((x[i] < x[j] and y[i] > y[j]) or (x[i] > x[j] and y[i] < y[j])): 
                disc += 1
            else:
                  continue
    res_conc = conc/2
    res_disc = disc / 2
    n = len(x)
    res = (res_conc - res_disc) / (n * (n - 1) / 2)
    return res, None

In [6]:
math_sub = ['мат. аналіз', 'лінійна алгебра','ятдр','дискр. математика','математична економіка', 
            'статистичні основи вебаналітики', 'обчислювальна геометрія', 'тймс', 'любий предмет міхалича']
test_1_data = data.loc[data['Назва предмету, який вважаєш проф-орієнтованим'].isin(math_sub)][['Test#1', 'Предмет#2']]
test_2_data = data[['Test#2: Гуманітарний предмет', 'Test#2: Аналіз даних']]
test_3_data = data[['Професійні мотиви', 'Соціальні мотиви']]

### Test1 та Оцінки з математики

In [8]:
x_1 = test_1_data['Test#1'].values
y_1 = test_1_data['Предмет#2'].values

In [9]:
corr_pearson_my, p_value_pearson_my = pearson(x_1, y_1)
corr_spearman_my, p_value_spearman_my = spearman(x_1, y_1)
corr_kendall_my, p_value_kendall_my = kendall(x_1, y_1)


print('My results')
print(f'Pearson corr   : {corr_pearson_my} p_value: {p_value_pearson_my}')
print(f'Spearman corr  : {corr_spearman_my} p_value: {p_value_spearman_my}')
print(f'Kendall corr   : {corr_kendall_my} p_value: {p_value_kendall_my}')

corr_pearson, p_value_pearson = stats.pearsonr(x_1, y_1)
corr_spearman, p_value_spearman = stats.spearmanr(x_1, y_1)
corr_kendall, p_value_kendall = stats.kendalltau(x_1, y_1)

print()
print(f'Pearson corr  : {corr_pearson} p_value: {p_value_pearson}')
print(f'Spearman corr : {corr_spearman} p_value: {p_value_spearman}')
print(f'Kendall corr  : {corr_kendall} p_value: {p_value_kendall}')

My results
Pearson corr   : 0.17667373207652126 p_value: 0.903637424639728
Spearman corr  : 0.17832349586218477 p_value: None
Kendall corr   : 0.11688311688311688 p_value: None

Pearson corr  : 0.17667373207652134 p_value: 0.19272515072054316
Spearman corr : 0.17832349586218477 p_value: 0.18853651129522364
Kendall corr  : 0.13318608789476324 p_value: 0.1907658497839645


### Test2 для математичного та гуманітрного предметів

In [10]:
x_2 = test_2_data['Test#2: Гуманітарний предмет'].values
y_2 = test_2_data['Test#2: Аналіз даних'].values

In [11]:
corr_pearson_my, p_value_pearson_my = pearson(x_2, y_2)
corr_spearman_my, p_value_spearman_my = spearman(x_2, y_2)
corr_kendall_my, p_value_kendall_my = kendall(x_2, y_2)


print('My results')
print(f'Pearson corr   : {corr_pearson_my} p_value: {p_value_pearson_my}')
print(f'Spearman corr  : {corr_spearman_my} p_value: {p_value_spearman_my}')
print(f'Kendall corr   : {corr_kendall_my} p_value: {p_value_kendall_my}')

corr_pearson, p_value_pearson = stats.pearsonr(x_2, y_2)
corr_spearman, p_value_spearman = stats.spearmanr(x_2, y_2)
corr_kendall, p_value_kendall = stats.kendalltau(x_2, y_2)

print()
print(f'Pearson corr  : {corr_pearson} p_value: {p_value_pearson}')
print(f'Spearman corr : {corr_spearman} p_value: {p_value_spearman}')
print(f'Kendall corr  : {corr_kendall} p_value: {p_value_kendall}')

My results
Pearson corr   : -0.1903838566857672 p_value: 0.027035854514929236
Spearman corr  : -0.22297112757003354 p_value: None
Kendall corr   : -0.14962878355225587 p_value: None

Pearson corr  : -0.19038385668576724 p_value: 0.054071709029858576
Spearman corr : -0.22297112757003357 p_value: 0.023581923556763123
Kendall corr  : -0.17362512473050165 p_value: 0.02137301745649448


### Test3 Професійні мотиви та Соціальні мотиви

In [12]:
x_3 = test_3_data['Професійні мотиви'].values
y_3 = test_3_data['Соціальні мотиви'].values

In [13]:
corr_pearson_my, p_value_pearson_my = pearson(x_3, y_3)
corr_spearman_my, p_value_spearman_my = spearman(x_3, y_3)
corr_kendall_my, p_value_kendall_my = kendall(x_3, y_3)


print('My results')
print(f'Pearson corr   : {corr_pearson_my} p_value: {p_value_pearson_my}')
print(f'Spearman corr  : {corr_spearman_my} p_value: {p_value_spearman_my}')
print(f'Kendall corr   : {corr_kendall_my} p_value: {p_value_kendall_my}')

corr_pearson, p_value_pearson = stats.pearsonr(x_3, y_3)
corr_spearman, p_value_spearman = stats.spearmanr(x_3, y_3)
corr_kendall, p_value_kendall = stats.kendalltau(x_3, y_3)

print()
print(f'Pearson corr  : {corr_pearson} p_value: {p_value_pearson}')
print(f'Spearman corr : {corr_spearman} p_value: {p_value_spearman}')
print(f'Kendall corr  : {corr_kendall} p_value: {p_value_kendall}')

My results
Pearson corr   : -0.003116304895110254 p_value: 0.48753861999535164
Spearman corr  : 0.00460872217907573 p_value: None
Kendall corr   : 0.011422044545973729 p_value: None

Pearson corr  : -0.003116304895110257 p_value: 0.9750772399907084
Spearman corr : 0.004608722179075731 p_value: 0.9631485887775919
Kendall corr  : 0.012391683138849504 p_value: 0.8627881806857811
